<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Facility_Location_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Facility Location Problem**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data

!pip install scgraph==2.1.0         # Python package used to compute paths and distances on a real-world transportation network
!pip install scgraph_data==2.0.0    # Python package used to compute paths and distances on a real-world transportation network


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.1/944.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.3 MB/s eta 0:00:00


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points

import scgraph                                                  # For computing paths and distances on a real-world transportation network
from scgraph.geographs.us_freeway import us_freeway_geograph    # Data on US highway network (for road paths and distances)
from scgraph.geographs.marnet import marnet_geograph            # Data on maritime routes (for ocean paths and distances)


## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing the shortest path between two points on a real road or ocean network**

In [ ]:
# Function for computing the shortest path between two points on a real road or ocean network

def shortest_path(origin, destination, mode, result, unit='mi'):

    # Extract coordinates from origin and destination objects
    origin_coordinates = (origin.lat, origin.lon)
    destination_coordinates = (destination.lat, destination.lon)

    # Calculate the shortest path on the ocean network
    if mode == 'ocean':
        output = marnet_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Calculate the shortest path on the road network
    elif mode == 'road':
        output = us_freeway_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Return the total distance of the path
    if result == 'distance':
        return output['length']

    # Return the coordinates representing the path
    elif result == 'coordinate_path':
        return output['coordinate_path']


### **Plotting flows on a map**

In [ ]:
# Defining a function to plot flows on a map using folium

def plot_flows(map,                   # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the coordinates of the nodes for plotting the line
            points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


            # Add a PolyLine to the map to represent the flow between the nodes
            folium.PolyLine(points,
                            color=color,                                # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map


### **Plotting flows (real network) on the map**

In [ ]:
# Functions that adjust the arc path to ensure longitude continuity across the globe

def adjustArcPath(path):
    for index in range(1, len(path)):
        x = path[index][1]
        prevX = path[index - 1][1]
        path[index][1] = x - (round((x - prevX)/360,0) * 360)
    return path

def modifyArcPathLong(points, amount):
    return [[i[0], i[1]+amount] for i in points]

def getCleanArcPath(path):
    path = adjustArcPath(path)
    return [
        path,
        modifyArcPathLong(path, 360),
        modifyArcPathLong(path, -360),
        modifyArcPathLong(path, 720),
        modifyArcPathLong(path, -720)
    ]

# Plots real flows on a Folium map based on optimization model results

def plot_real_flows(map,              # Folium map object where flows will be plotted.
               vars,                  # Dictionary of flow decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               mode,                  # Transportation mode (road or ocean)
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the shortest path coordinates for the node pair
            path = shortest_path(nodes[node1_key], nodes[node2_key], mode = mode, result = 'coordinate_path')

            # Add the path as a polyline on the map, with width proportional to the flow value
            folium.PolyLine(getCleanArcPath(path),
                      color=color,
                      weight=var.X / max_val * max_width,   # Normalize weight by maximum flow value
                      opacity=opacity).add_to(map)


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/facility_location/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,lat,lon,population,demand
0,c1,Chicago Ridge,41.7034,-87.7795,1860281,6400
1,c2,Los Osos,35.3065,-120.8242,809268,6400
2,c3,Carneys Point,39.6967,-75.4475,4040,6400
3,c4,Pompano Beach,26.2428,-80.1312,1743000,6400
4,c5,Collinsville,38.6770,-90.0063,438341,6400


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/facility_location/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()

,ID,name,lat,lon,supply
0,s1,Chicago,41.8375,-87.6866,39000
1,s2,Tampa,27.9945,-82.4447,39000
2,s3,Boston,42.3188,-71.0852,39000
3,s4,Sarasota,27.3387,-82.5432,39000
4,s5,Aberdeen,40.4165,-74.2249,39000


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/facility_location/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,lat,lon,capacity,fixed_cost
0,w1,Poinciana,28.1217,-81.4821,30000,5000000
1,w2,Huntington Beach,33.6960,-118.0018,30000,5000000
2,w3,Pryor Creek,36.2996,-95.3107,30000,5000000
3,w4,West Pensacola,30.4263,-87.2679,30000,5000000
4,w5,Blackstone,42.0399,-71.5313,30000,5000000


#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon, demand):
        self.ID = ID              # Customer's ID
        self.name = name          # Customer's name
        self.lat = lat            # Customer's latitude
        self.lon = lon            # Customer's longitude
        self.demand = demand      # Customer's demand

        self.active = True        # Initializing node as active


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, supply):
        self.ID = ID            # Supplier's ID
        self.name = name        # Supplier's name
        self.lat = lat          # Supplier's latitude
        self.lon = lon          # Supplier's longitude
        self.supply = supply    # Supplier's available supply

        self.active = True      # Initializing node as active

In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, capacity, fixed_cost):
        self.ID = ID                  # Warehouse's ID
        self.name = name              # Warehouse's name
        self.lat = lat                # Warehouse's latitude
        self.lon = lon                # Warehouse's longitude
        self.capacity = capacity      # Warehouse's capacity
        self.fixed_cost = fixed_cost  # Warehouse's fixed cost

        self.active = True            # Initializing node as active



#### Creating node objects

In [ ]:
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'],         # Customer's longitude
                                    demand=row['demand'])   # Customer's demand

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'],         # Supplier's longitude
                                    supply=row['supply'])   # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**customers, **suppliers}

In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],               # Warehouse's ID
                                    name = row['name'],             # Warehouse's name
                                    lat = row['lat'],               # Warehouse's latitude
                                    lon = row['lon'],               # Warehouse's longitude
                                    capacity = row['capacity'],     # Warehouse's capacity
                                    fixed_cost = row['fixed_cost']) # Warehouse's fixed cost

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

#### Visualizing node objects

In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between suppliers and warehouses and warehouses and customers

distances = dict()
for s, supplier in suppliers.items():
  for w, warehouse in warehouses.items():
      distances[s, w] = shortest_path(origin = supplier, destination = warehouse, mode = 'road', result = 'distance', unit = 'km')

for w, warehouse in warehouses.items():
  for c, customer in customers.items():
      distances[w, c] = shortest_path(origin = warehouse, destination = customer, mode = 'road', result = 'distance', unit = 'km')

#### Arc costs

In [ ]:
cost_unit_km_supplier_warehouse = 0.2 # Cost per unit per kilometer from supplier to warehouse
cost_unit_km_warehouse_customer = 0.3 # Cost per unit per kilometer from warehouse to customer

# Creating a dictionary containing unit costs between suppliers and warehouses, and between warehouses and customers
unit_cost = dict()

for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    for w, warehouse in warehouses.items():                                                    # Iterate over warehouses
        unit_cost[s, w] = distances[s, w] * cost_unit_km_supplier_warehouse                    # Calculate unit cost as distance multiplied by cost per km (supplier to warehouse)

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        unit_cost[w, c] = distances[w, c] * cost_unit_km_warehouse_customer                    # Calculate unit cost as distance multiplied by cost per km (warehouse to customer)


## **Optimization model**

### **Creating and solving the optimization model**

In [ ]:
# Initializing the optimization model for the Facility Location Problem
model = grb.Model("Facility Location Problem")

# Creating decision variables

# Decision variable representing the flows between suppliers and warehouses, and warehouses and customers
flow_vars = dict()
for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    for w, warehouse in warehouses.items():                                                    # Iterate over warehouses
        flow_vars[s, w] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_vars_{0}_{1}".format(s, w))

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        flow_vars[w, c] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_vars_{0}_{1}".format(w, c))

# Binary decision variables to determine if a warehouse is opened
wh_location_vars = dict()
for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                               name = "wh_location_vars{0}".format(w))

# Creating the objective function

# Cost for flows from suppliers to warehouses
cost_supplier_warehouse = grb.quicksum(unit_cost[s, w] * flow_vars[s, w]
                                       for s, supplier in suppliers.items()
                                       for w, warehouse in warehouses.items())

# Cost for flows from warehouses to customers
cost_warehouse_customer = grb.quicksum(unit_cost[w, c] * flow_vars[w, c]
                                       for w, warehouse in warehouses.items()
                                       for c, customer in customers.items())

# Fixed cost for opening warehouses
fixed_cost_warehouse = grb.quicksum(wh_location_vars[w] * warehouses[w].fixed_cost
                                       for w, warehouse in warehouses.items())

# Total cost is the sum of flow costs and fixed warehouse costs
total_cost = cost_supplier_warehouse + cost_warehouse_customer + fixed_cost_warehouse

# Setting the objective to minimize total cost
model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Adding demand constraints
for c, customer in customers.items():                                                          # Iterate over customers
    model.addConstr(grb.quicksum(flow_vars[w, c] for w, warehouse in warehouses.items()) == customer.demand)  # Flow into customer must meet demand

# Adding supply constraints
for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    model.addConstr(grb.quicksum(flow_vars[s, w] for w, warehouse in warehouses.items()) <= supplier.supply)  # Flow out of supplier cannot exceed supply

# Adding flow preservation constraints
for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    model.addConstr(grb.quicksum(flow_vars[s, w] for s, supplier in suppliers.items()) ==       # Flow into warehouse must equal flow out
                    grb.quicksum(flow_vars[w, c] for c, customer in customers.items()))

# Adding warehouse capacity constraints (flow into warehouse must not exceed capacity if it is open)
for w, warehouse in warehouses.items():
    model.addConstr(grb.quicksum(flow_vars[s, w] for s in suppliers) <= warehouse.capacity * wh_location_vars[w])

# Solving the optimization model
model.optimize()

# Output results
print('Fixed Cost Warehouses: ', fixed_cost_warehouse.getValue())
print('Number of Active Warehouses: ', grb.quicksum(wh_location_vars[w] for w in warehouses.keys()).getValue())

# Updating warehouse status based on optimization results
for w, warehouse in warehouses.items():
  if wh_location_vars[w].X == 1:  # If the warehouse is selected
    warehouse.active = True
  else:                           # If the warehouse is not selected
    warehouse.active = False

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 71 rows, 646 columns and 1394 nonzeros
Model fingerprint: 0x7d087104
Variable types: 629 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [8e+00, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+03, 4e+04]
Found heuristic solution: objective 2.574116e+08
Presolve time: 0.00s
Presolved: 71 rows, 646 columns, 1394 nonzeros
Variable types: 629 continuous, 17 integer (17 binary)

Root relaxation: objective 9.166424e+07, 86 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

## **Solution visualization and analysis**

In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Plot the flows with a maximum line width of 20
plot_flows(map=map, max_width=20, vars=flow_vars, nodes = nodes)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map



In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Plot the flows with a maximum line width of 20
plot_real_flows(map=map, max_width=20, vars=flow_vars, mode = 'road', nodes = nodes)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map

